## K-NN Classifier

In [1]:
from init import *

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [3]:
%matplotlib inline

In [4]:
X = train_features.reshape(50000, 3*32*32)
Xt = test_features.reshape(10000, 3*32*32)
y = train_labels.flatten()
yt = test_labels.flatten()

In [5]:
msg("[K-NN] Training")
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X, y)
done()

[K-NN] Training ... done.


In [6]:
msg("[K-NN] Testing Accuracy")

count = 0
def predict_batch(x):
    global count
    count += 25
    print("\rClassifiying {} ... ".format(count), end="")
    return knn.predict(x)

Xt_pred = np.array([predict_batch(Xt[i*25:(i+1)*25]) for i in range(400)]).reshape(-1, 1)
# Xt_pred = np.apply_along_axis(predict_with_update, 1, Xt).reshape(-1, 1)
# Xt_pred = knn.predict(Xt)
msg(metrics.accuracy_score(yt, Xt_pred))
done()

Classifiying 10000 ... 0.3386 ... done.


In [9]:
import time
class NearestNeighbor(object):
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)
    
    start = time.time()
    # loop over all test rows
    for i in range(num_test):
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
      elapsed = time.time()-start
      print("\rClassified {} ... Elapsed Time: {}m {}s".format(i, int(elapsed/60), int(elapsed%60)),end="")

    return Ypred

nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(X, y) # train the classifier on the training images and labels
Xt_predict = nn.predict(Xt) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print('accuracy: %f' % ( metrics.accuracy_score(yt, Xt_predict)))

Classified 9999 ... Elapsed Time: 79m 44saccuracy: 0.249200
